<a href="https://colab.research.google.com/github/ashaaher/Emerging-Technologies/blob/master/Assignment%204/Relation_Network_Using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Relation Network Using Tensorflow

We are considering simple binary classification problem in this collab and see how can we solve them using a relation network.

In [1]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 56kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 3.1MB 51.6MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

1.12.0


Importing Required Libraries

In [3]:
import tensorflow as tf
import numpy as np

Randomly generating some 1000 data points for each of these classes

In [4]:
classA = np.random.rand(1000,18)

ClassB = np.random.rand(1000,18)

Creating dataset by combining both of these classes

In [5]:
data = np.vstack([classA, ClassB])


Setting the label,assigning label 1 for classA and label 0 for classB.

In [6]:
label = np.vstack([np.ones((len(classA),1)),np.zeros((len(ClassB),1))])

In [7]:
data.shape

(2000, 18)

In [8]:
label.shape

(2000, 1)

Placeholders for  support set xi and query set xj

In [9]:
xi = tf.placeholder(tf.float32, [None, 9])
xj = tf.placeholder(tf.float32, [None, 9])

Defining the placeholder for label y

In [10]:
y = tf.placeholder(tf.float32, [None, 1])

Embedding function to learn the embedding of support set and query set. Using a normal feedforward network as our embedding function.

In [11]:
def embedding_function(x):
    
    weights = tf.Variable(tf.truncated_normal([9,1]))
    bias = tf.Variable(tf.truncated_normal([1]))
    
    a = (tf.nn.xw_plus_b(x,weights,bias))
    embeddings = tf.nn.relu(a)
    
    return embeddings


Computing the embeddings of support set xi

In [12]:
f_xi = embedding_function(xi)


Computing the embeddings of query set 

In [13]:
f_xj = embedding_function(xj)

Combining both the support set and query set feature vectors 

In [14]:
Z = tf.concat([f_xi,f_xj],axis=1)

defining  relation function as three layered neural network with relu activations

In [15]:
def relation_function(x):
    w1 = tf.Variable(tf.truncated_normal([2,3]))
    b1 = tf.Variable(tf.truncated_normal([3]))
    
    w2 = tf.Variable(tf.truncated_normal([3,5]))
    b2 = tf.Variable(tf.truncated_normal([5]))
    
    w3 = tf.Variable(tf.truncated_normal([5,1]))
    b3 = tf.Variable(tf.truncated_normal([1]))
    
    #layer1
    z1 = (tf.nn.xw_plus_b(x,w1,b1))
    a1 = tf.nn.relu(z1)
    
    #layer2
    z2 = tf.nn.xw_plus_b(a1,w2,b2)
    a2 = tf.nn.relu(z2)
    
    #layer3
    z3 = tf.nn.xw_plus_b(z2,w3,b3)

    #output
    y = tf.nn.sigmoid(z3)
    
    return y

Passing the concatenated feature vectors of support set and query set to the relation function and get the relation scores

In [16]:
relation_scores = relation_function(Z)

Defining loss function as mean squared error i.e squared difference between relation scores and actual y value.

In [17]:
loss_function = tf.reduce_mean(tf.squared_difference(relation_scores,y))

Minimizing the loss using adam optimizer

In [18]:
optimizer = tf.train.AdamOptimizer(0.1)
train = optimizer.minimize(loss_function)

Starting tensorflow session

In [19]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Randomly sample data points for  support set xi and query set xj and train the network.

In [20]:
for episode in range(1000):
    _, loss_value = sess.run([train, loss_function], 
                             feed_dict={xi:data[:,0:9]+np.random.randn(*np.shape(data[:,0:9]))*0.05,
                                        xj:data[:,9:]+np.random.randn(*np.shape(data[:,9:]))*0.05,
                                        y:label})
    if episode % 100 == 0:
        print("Episode {}: loss {:.3f} ".format(episode, loss_value))

Episode 0: loss 0.362 
Episode 100: loss 0.250 
Episode 200: loss 0.250 
Episode 300: loss 0.250 
Episode 400: loss 0.250 
Episode 500: loss 0.250 
Episode 600: loss 0.250 
Episode 700: loss 0.250 
Episode 800: loss 0.250 
Episode 900: loss 0.250 


Reference: https://github.com/sudharsan13296/Hands-On-Meta-Learning-With-Python/blob/master/04.%20Relation%20and%20Matching%20Networks%20Using%20Tensorflow/4.5%20Building%20Relation%20Network%20Using%20Tensorflow.ipynb